In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.discrete.discrete_model as discrete_model

In [2]:
import statsmodels.discrete.discrete_model
import statsmodels.formula.api as smf

# Import

In [3]:
df02_mg = pd.read_stata("data/hh02dta_b3a/iiia_mg.dta", convert_categoricals = False)
df02_mg1 = pd.read_stata("data/hh02dta_b3a/iiia_mg1.dta", convert_categoricals = False)
df02_mt = pd.read_stata("data/hh02dta_b3a/iiia_mt.dta", convert_categoricals = False)
df02_mt1 = pd.read_stata("data/hh02dta_b3a/iiia_mt1.dta", convert_categoricals = False)
df02_ed = pd.read_stata("data/hh02dta_b3a/iiia_ed.dta", convert_categoricals = False)
df02_portad = pd.read_stata("data/hh02dta_b3a/iiia_portad.dta", convert_categoricals = False)
df02_hm1 = pd.read_stata("data/hh02dta_b3a/iiia_hm1.dta", convert_categoricals = False)

df02_ls = pd.read_stata("data/hh02dta_bc/c_ls.dta", convert_categoricals = False)

# Data Cleaning

## Temporary Migration for Job

In [4]:
c02_mt = df02_mt1[["folio", "ls", "mt14a"]]
c02_mt["mtJob"] = c02_mt["mt14a"].fillna(0)
c02_mt = c02_mt[["folio", "ls", "mtJob"]]

//anaconda3/envs/econ/lib/python3.7/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app


## Desire to Migrate

In [5]:
c02_desire = df02_mg[["folio", "ls", "mg36_1"]]
c02_desire["desire"] = (c02_desire["mg36_1"] == 1) * 1
c02_desire = c02_desire[["folio", "ls", "desire"]]

//anaconda3/envs/econ/lib/python3.7/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app


## Permanent Migration for Job

In [6]:
c02_mg = df02_mg1[["folio", "ls", "mg25a"]]

In [7]:
c02_mg["mg25a"] = c02_mg["mg25a"] * 1
c02_mg["mg25a"] = c02_mg["mg25a"].fillna(0)

//anaconda3/envs/econ/lib/python3.7/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':
//anaconda3/envs/econ/lib/python3.7/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app


In [8]:
c02_permanent = c02_mg[["folio", "ls"]].drop_duplicates(["folio", "ls"])

has_moved = (c02_mg.groupby(["folio", "ls"]).sum()["mg25a"].values) > 0

c02_permanent["mg_perm_job"] = has_moved * 1

### Education

In [9]:
c02_ed = df02_ed[["ed03", "ed06", "ed09", "ed14_22", "ed15_2", "ed17_2", "folio", "ls"]]
c02_ed = pd.merge(c02_ed, df02_portad[["folio", "ls", "edad", "ent"]], how = 'left', on = ['folio','ls'])


In [10]:

c02_ed = c02_ed.rename(columns={"ed03": "indig",
                                "ed09": "technical",
                                "ed14_22": "year_quit",
                                "ed15_2": "age_quit",
                                "ed17_2": "age_start"})

# Change from their coding to dummy
c02_ed["indig"] = (c02_ed["indig"] == 1)

# Change don't know codings to 0
c02_ed[c02_ed["ed06"] == 98]["ed06"] = 0

# Create some dummy variables
c02_ed["hs"] = (c02_ed["ed06"] >= 6)
c02_ed["college"] = (c02_ed["ed06"] >= 8)

# Calculate age_quit using current age and year_quit
c02_ed["age_quit"] = c02_ed["age_quit"].fillna(c02_ed["edad"] - (2002 - c02_ed["year_quit"]))
c02_ed["years_ed"] = c02_ed["age_quit"] - c02_ed["age_start"]

# Calculate "years_ed" (imputed) by filling with mean
c02_ed["age_quit"] = c02_ed["age_quit"].fillna(c02_ed["age_quit"].mean())
c02_ed["age_start"] = c02_ed["age_start"].fillna(c02_ed["age_start"].mean())

c02_ed["years_ed_impute"] = c02_ed["age_quit"] - c02_ed["age_start"]

c02_ed["technical"] = (c02_ed["technical"].fillna(0) == 1)

c02_ed = c02_ed[["folio", "ls", "indig", "hs", "college", "technical", "years_ed", "years_ed_impute", "edad", "ent"]]

//anaconda3/envs/econ/lib/python3.7/site-packages/ipykernel/__main__.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


## Marriage

In [11]:
c02_hm = df02_hm1[["folio", "ls", "hm08"]]

In [12]:
c02_hm["married"] = (c02_hm["hm08"] <= 2)
c02_hm = c02_hm[["folio", "ls", "married"]]

//anaconda3/envs/econ/lib/python3.7/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


## Household Size and Gender

In [13]:
c02_ls = df02_ls[["folio", "ls", "ls04"]]

In [14]:
c02_ls["male"] = (c02_ls["ls04"] == 1) * 1

# Calculate HH size by number of HH respondents in MXFLS
hh_size = df02_ls.groupby('folio')["folio"].count()
c02_ls["hh_size"] = c02_ls["folio"].apply(lambda x: hh_size[x])

c02_ls = c02_ls[["folio", "ls", "male", "hh_size"]]

//anaconda3/envs/econ/lib/python3.7/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':
//anaconda3/envs/econ/lib/python3.7/site-packages/ipykernel/__main__.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


# Regressions

In [15]:
dg_formula = " + male + edad + married + indig + hh_size"

## Desire to Migrate

In [16]:
regdf = c02_desire

regdf = pd.merge(regdf, c02_ed, how='inner', on = ['folio','ls'])
regdf = pd.merge(regdf, c02_hm, how='inner', on = ['folio','ls'])
regdf = pd.merge(regdf, c02_ls, how='inner', on = ['folio','ls'])

ent_dummies = pd.get_dummies(regdf["ent"])
ent_dummies.columns = ["ent_" + str(col).split(".")[0] for col in ent_dummies.columns]
regdf = pd.concat((regdf, ent_dummies), axis = 1)

regdf = regdf.drop_duplicates()

In [17]:
ent_dummies_formula = ''

for col in ent_dummies.columns:
    ent_dummies_formula += (" + %s" % col)

print(ent_dummies_formula)

 + ent_3 + ent_5 + ent_9 + ent_10 + ent_11 + ent_14 + ent_15 + ent_16 + ent_17 + ent_19 + ent_20 + ent_21 + ent_25 + ent_26 + ent_30 + ent_31


#### (A1)

In [18]:
reg = smf.probit('desire ~ hs' + dg_formula, data = regdf).fit()
print(reg.summary())
print(reg.get_margeff().summary())

Optimization terminated successfully.
         Current function value: 0.270368
         Iterations 7
                          Probit Regression Results                           
Dep. Variable:                 desire   No. Observations:                14956
Model:                         Probit   Df Residuals:                    14949
Method:                           MLE   Df Model:                            6
Date:                Sun, 12 Jan 2020   Pseudo R-squ.:                 0.07422
Time:                        22:28:08   Log-Likelihood:                -4043.6
converged:                       True   LL-Null:                       -4367.8
Covariance Type:            nonrobust   LLR p-value:                8.683e-137
                      coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------
Intercept          -0.3902      0.076     -5.115      0.000      -0.540      -0.241
hs[T.True]    

#### (A2)

In [19]:
reg = smf.probit('desire ~ hs' + dg_formula + ent_dummies_formula, data = regdf).fit()
print(reg.summary())
print(reg.get_margeff().summary())

Optimization terminated successfully.
         Current function value: 0.266422
         Iterations 7
                          Probit Regression Results                           
Dep. Variable:                 desire   No. Observations:                14956
Model:                         Probit   Df Residuals:                    14933
Method:                           MLE   Df Model:                           22
Date:                Sun, 12 Jan 2020   Pseudo R-squ.:                 0.08773
Time:                        22:28:09   Log-Likelihood:                -3984.6
converged:                       True   LL-Null:                       -4367.8
Covariance Type:            nonrobust   LLR p-value:                7.453e-148
                      coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------
Intercept          -0.0900      0.223     -0.403      0.687      -0.527       0.347
hs[T.True]    

#### (A3)

In [20]:
reg = smf.probit('desire ~ hs + college' + dg_formula, data = regdf).fit()
print(reg.summary())
print(reg.get_margeff().summary())

Optimization terminated successfully.
         Current function value: 0.270158
         Iterations 7
                          Probit Regression Results                           
Dep. Variable:                 desire   No. Observations:                14956
Model:                         Probit   Df Residuals:                    14948
Method:                           MLE   Df Model:                            7
Date:                Sun, 12 Jan 2020   Pseudo R-squ.:                 0.07494
Time:                        22:28:09   Log-Likelihood:                -4040.5
converged:                       True   LL-Null:                       -4367.8
Covariance Type:            nonrobust   LLR p-value:                4.192e-137
                      coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------
Intercept          -0.3768      0.077     -4.925      0.000      -0.527      -0.227
hs[T.True]    

#### (A4)

In [21]:
reg = smf.probit('desire ~ hs + college' + dg_formula + ent_dummies_formula, data = regdf).fit()
print(reg.summary())
print(reg.get_margeff().summary())

Optimization terminated successfully.
         Current function value: 0.266169
         Iterations 7
                          Probit Regression Results                           
Dep. Variable:                 desire   No. Observations:                14956
Model:                         Probit   Df Residuals:                    14932
Method:                           MLE   Df Model:                           23
Date:                Sun, 12 Jan 2020   Pseudo R-squ.:                 0.08860
Time:                        22:28:09   Log-Likelihood:                -3980.8
converged:                       True   LL-Null:                       -4367.8
Covariance Type:            nonrobust   LLR p-value:                1.117e-148
                      coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------
Intercept          -0.0703      0.223     -0.315      0.753      -0.508       0.367
hs[T.True]    

#### (A5)

In [22]:
reg = smf.probit('desire ~ technical' + dg_formula, data = regdf).fit()
print(reg.summary())
print(reg.get_margeff().summary())

Optimization terminated successfully.
         Current function value: 0.270819
         Iterations 7
                          Probit Regression Results                           
Dep. Variable:                 desire   No. Observations:                14956
Model:                         Probit   Df Residuals:                    14949
Method:                           MLE   Df Model:                            6
Date:                Sun, 12 Jan 2020   Pseudo R-squ.:                 0.07267
Time:                        22:28:10   Log-Likelihood:                -4050.4
converged:                       True   LL-Null:                       -4367.8
Covariance Type:            nonrobust   LLR p-value:                7.063e-134
                        coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------
Intercept            -0.3470      0.075     -4.602      0.000      -0.495      -0.199
technica

#### (A6)

In [23]:
reg = smf.probit('desire ~ technical' + dg_formula + ent_dummies_formula, data = regdf).fit()
print(reg.summary())
print(reg.get_margeff().summary())

Optimization terminated successfully.
         Current function value: 0.266767
         Iterations 7
                          Probit Regression Results                           
Dep. Variable:                 desire   No. Observations:                14956
Model:                         Probit   Df Residuals:                    14933
Method:                           MLE   Df Model:                           22
Date:                Sun, 12 Jan 2020   Pseudo R-squ.:                 0.08655
Time:                        22:28:10   Log-Likelihood:                -3989.8
converged:                       True   LL-Null:                       -4367.8
Covariance Type:            nonrobust   LLR p-value:                1.131e-145
                        coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------
Intercept            -0.0800      0.222     -0.360      0.719      -0.516       0.356
technica

#### (A7)

In [24]:
reg = smf.probit('desire ~ hs + college + technical' + dg_formula, data = regdf).fit()
print(reg.summary())
print(reg.get_margeff().summary())

Optimization terminated successfully.
         Current function value: 0.268916
         Iterations 7
                          Probit Regression Results                           
Dep. Variable:                 desire   No. Observations:                14956
Model:                         Probit   Df Residuals:                    14947
Method:                           MLE   Df Model:                            8
Date:                Sun, 12 Jan 2020   Pseudo R-squ.:                 0.07919
Time:                        22:28:11   Log-Likelihood:                -4021.9
converged:                       True   LL-Null:                       -4367.8
Covariance Type:            nonrobust   LLR p-value:                4.226e-144
                        coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------
Intercept            -0.4370      0.077     -5.645      0.000      -0.589      -0.285
hs[T.Tru

#### (A8)

In [25]:
reg = smf.probit('desire ~ hs + college + technical' + dg_formula + ent_dummies_formula, data = regdf).fit()
print(reg.summary())
print(reg.get_margeff().summary())

Optimization terminated successfully.
         Current function value: 0.264883
         Iterations 7
                          Probit Regression Results                           
Dep. Variable:                 desire   No. Observations:                14956
Model:                         Probit   Df Residuals:                    14931
Method:                           MLE   Df Model:                           24
Date:                Sun, 12 Jan 2020   Pseudo R-squ.:                 0.09300
Time:                        22:28:11   Log-Likelihood:                -3961.6
converged:                       True   LL-Null:                       -4367.8
Covariance Type:            nonrobust   LLR p-value:                4.970e-156
                        coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------
Intercept            -0.1416      0.223     -0.635      0.525      -0.578       0.295
hs[T.Tru

#### (A9)

In [26]:
reg = smf.probit('desire ~ years_ed' + dg_formula, data = regdf).fit()
print(reg.summary())
print(reg.get_margeff().summary())

Optimization terminated successfully.
         Current function value: 0.290963
         Iterations 7
                          Probit Regression Results                           
Dep. Variable:                 desire   No. Observations:                12088
Model:                         Probit   Df Residuals:                    12081
Method:                           MLE   Df Model:                            6
Date:                Sun, 12 Jan 2020   Pseudo R-squ.:                 0.06013
Time:                        22:28:12   Log-Likelihood:                -3517.2
converged:                       True   LL-Null:                       -3742.2
Covariance Type:            nonrobust   LLR p-value:                 4.797e-94
                      coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------
Intercept          -0.5710      0.087     -6.596      0.000      -0.741      -0.401
married[T.True

#### (A10)

In [27]:
reg = smf.probit('desire ~ years_ed' + dg_formula + ent_dummies_formula, data = regdf).fit()
print(reg.summary())
print(reg.get_margeff().summary())

Optimization terminated successfully.
         Current function value: 0.286302
         Iterations 7
                          Probit Regression Results                           
Dep. Variable:                 desire   No. Observations:                12088
Model:                         Probit   Df Residuals:                    12065
Method:                           MLE   Df Model:                           22
Date:                Sun, 12 Jan 2020   Pseudo R-squ.:                 0.07519
Time:                        22:28:12   Log-Likelihood:                -3460.8
converged:                       True   LL-Null:                       -3742.2
Covariance Type:            nonrobust   LLR p-value:                5.713e-105
                      coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------
Intercept          -0.3835      0.251     -1.527      0.127      -0.876       0.109
married[T.True

#### (A11)

In [28]:
reg = smf.probit('desire ~ years_ed_impute' + dg_formula, data = regdf).fit()
print(reg.summary())
print(reg.get_margeff().summary())

Optimization terminated successfully.
         Current function value: 0.270122
         Iterations 7
                          Probit Regression Results                           
Dep. Variable:                 desire   No. Observations:                14956
Model:                         Probit   Df Residuals:                    14949
Method:                           MLE   Df Model:                            6
Date:                Sun, 12 Jan 2020   Pseudo R-squ.:                 0.07506
Time:                        22:28:12   Log-Likelihood:                -4039.9
converged:                       True   LL-Null:                       -4367.8
Covariance Type:            nonrobust   LLR p-value:                2.245e-138
                      coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------
Intercept          -0.4858      0.079     -6.150      0.000      -0.641      -0.331
married[T.True

#### (A12)

In [29]:
reg = smf.probit('desire ~ years_ed_impute' + dg_formula + ent_dummies_formula, data = regdf).fit()
print(reg.summary())
print(reg.get_margeff().summary())

Optimization terminated successfully.
         Current function value: 0.266062
         Iterations 7
                          Probit Regression Results                           
Dep. Variable:                 desire   No. Observations:                14956
Model:                         Probit   Df Residuals:                    14933
Method:                           MLE   Df Model:                           22
Date:                Sun, 12 Jan 2020   Pseudo R-squ.:                 0.08896
Time:                        22:28:13   Log-Likelihood:                -3979.2
converged:                       True   LL-Null:                       -4367.8
Covariance Type:            nonrobust   LLR p-value:                3.960e-150
                      coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------
Intercept          -0.1987      0.224     -0.885      0.376      -0.639       0.241
married[T.True

## Temporary Migration

In [30]:
regdf = c02_ed

regdf = pd.merge(regdf, c02_mt, how='left', on = ['folio','ls'])
regdf["mtJob"] = regdf["mtJob"].fillna(0)
regdf = pd.merge(regdf, c02_hm, how='inner', on = ['folio','ls'])
regdf = pd.merge(regdf, c02_ls, how='inner', on = ['folio','ls'])

ent_dummies = pd.get_dummies(regdf["ent"])
ent_dummies.columns = ["ent_" + str(col).split(".")[0] for col in ent_dummies.columns]
regdf = pd.concat((regdf, ent_dummies), axis = 1)

regdf = regdf.drop_duplicates()

In [31]:
ent_dummies_formula = ''

for col in ent_dummies.columns:
    ent_dummies_formula += (" + %s" % col)

print(ent_dummies_formula)

 + ent_3 + ent_5 + ent_9 + ent_10 + ent_11 + ent_14 + ent_15 + ent_16 + ent_17 + ent_19 + ent_20 + ent_21 + ent_25 + ent_26 + ent_30 + ent_31


#### (B1)

In [32]:
reg = smf.probit('mtJob ~ hs' + dg_formula, data = regdf).fit()
print(reg.summary())
print(reg.get_margeff().summary())

Optimization terminated successfully.
         Current function value: 0.066986
         Iterations 9
                          Probit Regression Results                           
Dep. Variable:                  mtJob   No. Observations:                14972
Model:                         Probit   Df Residuals:                    14965
Method:                           MLE   Df Model:                            6
Date:                Sun, 12 Jan 2020   Pseudo R-squ.:                  0.1319
Time:                        22:28:14   Log-Likelihood:                -1002.9
converged:                       True   LL-Null:                       -1155.2
Covariance Type:            nonrobust   LLR p-value:                 8.144e-63
                      coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------
Intercept          -1.7105      0.142    -12.052      0.000      -1.989      -1.432
hs[T.True]    

#### (B2)

In [33]:
reg = smf.probit('mtJob ~ hs' + dg_formula + ent_dummies_formula, data = regdf).fit()
print(reg.summary())
print(reg.get_margeff().summary())

         Current function value: 0.065237
         Iterations: 35
                          Probit Regression Results                           
Dep. Variable:                  mtJob   No. Observations:                14972
Model:                         Probit   Df Residuals:                    14949
Method:                           MLE   Df Model:                           22
Date:                Sun, 12 Jan 2020   Pseudo R-squ.:                  0.1545
Time:                        22:28:14   Log-Likelihood:                -976.72
converged:                      False   LL-Null:                       -1155.2
Covariance Type:            nonrobust   LLR p-value:                 2.823e-62
                      coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------
Intercept          -5.6395    399.397     -0.014      0.989    -788.443     777.164
hs[T.True]          0.0212      0.073      0.290  

//anaconda3/envs/econ/lib/python3.7/site-packages/statsmodels/base/model.py:512: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)


       Probit Marginal Effects       
Dep. Variable:                  mtJob
Method:                          dydx
At:                           overall
                     dy/dx    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------
hs[T.True]          0.0007      0.002      0.290      0.772      -0.004       0.005
married[T.True]    -0.0098      0.003     -3.860      0.000      -0.015      -0.005
indig[T.True]       0.0093      0.003      3.219      0.001       0.004       0.015
male                0.0325      0.003     10.849      0.000       0.027       0.038
edad               -0.0007   8.27e-05     -8.047      0.000      -0.001      -0.001
hh_size            -0.0006      0.000     -1.216      0.224      -0.002       0.000
ent_3               0.1266     13.081      0.010      0.992     -25.513      25.766
ent_5               0.1220     13.081      0.009      0.993     -25.517      25.761
ent_9   

#### (B3)

In [34]:
reg = smf.probit('mtJob ~ hs + college' + dg_formula, data = regdf).fit()
print(reg.summary())
print(reg.get_margeff().summary())

Optimization terminated successfully.
         Current function value: 0.066983
         Iterations 9
                          Probit Regression Results                           
Dep. Variable:                  mtJob   No. Observations:                14972
Model:                         Probit   Df Residuals:                    14964
Method:                           MLE   Df Model:                            7
Date:                Sun, 12 Jan 2020   Pseudo R-squ.:                  0.1319
Time:                        22:28:15   Log-Likelihood:                -1002.9
converged:                       True   LL-Null:                       -1155.2
Covariance Type:            nonrobust   LLR p-value:                 5.812e-62
                      coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------
Intercept          -1.7130      0.142    -12.046      0.000      -1.992      -1.434
hs[T.True]    

#### (B4)

In [35]:
reg = smf.probit('mtJob ~ hs + college' + dg_formula + ent_dummies_formula, data = regdf).fit()
print(reg.summary())
print(reg.get_margeff().summary())

         Current function value: 0.065237
         Iterations: 35
                          Probit Regression Results                           
Dep. Variable:                  mtJob   No. Observations:                14972
Model:                         Probit   Df Residuals:                    14948
Method:                           MLE   Df Model:                           23
Date:                Sun, 12 Jan 2020   Pseudo R-squ.:                  0.1545
Time:                        22:28:16   Log-Likelihood:                -976.72
converged:                      False   LL-Null:                       -1155.2
Covariance Type:            nonrobust   LLR p-value:                 1.153e-61
                      coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------
Intercept          -5.5920    352.150     -0.016      0.987    -695.794     684.610
hs[T.True]          0.0196      0.092      0.212  

//anaconda3/envs/econ/lib/python3.7/site-packages/statsmodels/base/model.py:512: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)


       Probit Marginal Effects       
Dep. Variable:                  mtJob
Method:                          dydx
At:                           overall
                     dy/dx    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------
hs[T.True]          0.0006      0.003      0.212      0.832      -0.005       0.007
college[T.True]     0.0001      0.004      0.028      0.977      -0.008       0.008
married[T.True]    -0.0098      0.003     -3.860      0.000      -0.015      -0.005
indig[T.True]       0.0093      0.003      3.219      0.001       0.004       0.015
male                0.0325      0.003     10.849      0.000       0.027       0.038
edad               -0.0007   8.31e-05     -8.009      0.000      -0.001      -0.001
hh_size            -0.0006      0.000     -1.214      0.225      -0.002       0.000
ent_3               0.1250     11.534      0.011      0.991     -22.481      22.731
ent_5   

#### (B5)

In [36]:
reg = smf.probit('mtJob ~ technical' + dg_formula, data = regdf).fit()
print(reg.summary())
print(reg.get_margeff().summary())

Optimization terminated successfully.
         Current function value: 0.066933
         Iterations 9
                          Probit Regression Results                           
Dep. Variable:                  mtJob   No. Observations:                14972
Model:                         Probit   Df Residuals:                    14965
Method:                           MLE   Df Model:                            6
Date:                Sun, 12 Jan 2020   Pseudo R-squ.:                  0.1325
Time:                        22:28:16   Log-Likelihood:                -1002.1
converged:                       True   LL-Null:                       -1155.2
Covariance Type:            nonrobust   LLR p-value:                 3.766e-63
                        coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------
Intercept            -1.6731      0.140    -11.933      0.000      -1.948      -1.398
technica

#### (B6)

In [37]:
reg = smf.probit('mtJob ~ technical' + dg_formula + ent_dummies_formula, data = regdf).fit()
print(reg.summary())
print(reg.get_margeff().summary())

         Current function value: 0.065177
         Iterations: 35
                          Probit Regression Results                           
Dep. Variable:                  mtJob   No. Observations:                14972
Model:                         Probit   Df Residuals:                    14949
Method:                           MLE   Df Model:                           22
Date:                Sun, 12 Jan 2020   Pseudo R-squ.:                  0.1553
Time:                        22:28:17   Log-Likelihood:                -975.83
converged:                      False   LL-Null:                       -1155.2
Covariance Type:            nonrobust   LLR p-value:                 1.216e-62
                        coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------
Intercept            -5.1596    127.298     -0.041      0.968    -254.659     244.340
technical[T.True]    -0.1349      0.101     

//anaconda3/envs/econ/lib/python3.7/site-packages/statsmodels/base/model.py:512: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)


       Probit Marginal Effects       
Dep. Variable:                  mtJob
Method:                          dydx
At:                           overall
                       dy/dx    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------
technical[T.True]    -0.0044      0.003     -1.331      0.183      -0.011       0.002
married[T.True]      -0.0097      0.003     -3.842      0.000      -0.015      -0.005
indig[T.True]         0.0091      0.003      3.143      0.002       0.003       0.015
male                  0.0326      0.003     10.866      0.000       0.027       0.038
edad                 -0.0007   8.21e-05     -8.224      0.000      -0.001      -0.001
hh_size              -0.0006      0.000     -1.333      0.183      -0.002       0.000
ent_3                 0.1123      4.166      0.027      0.978      -8.053       8.278
ent_5                 0.1074      4.166      0.026      0.979      -8.058 

#### (B7)

In [38]:
reg = smf.probit('mtJob ~ hs + college + technical' + dg_formula, data = regdf).fit()
print(reg.summary())
print(reg.get_margeff().summary())

Optimization terminated successfully.
         Current function value: 0.066917
         Iterations 9
                          Probit Regression Results                           
Dep. Variable:                  mtJob   No. Observations:                14972
Model:                         Probit   Df Residuals:                    14963
Method:                           MLE   Df Model:                            8
Date:                Sun, 12 Jan 2020   Pseudo R-squ.:                  0.1328
Time:                        22:28:17   Log-Likelihood:                -1001.9
converged:                       True   LL-Null:                       -1155.2
Covariance Type:            nonrobust   LLR p-value:                 1.528e-61
                        coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------
Intercept            -1.6908      0.143    -11.820      0.000      -1.971      -1.410
hs[T.Tru

#### (B8)

In [39]:
reg = smf.probit('mtJob ~ hs + college + technical' + dg_formula + ent_dummies_formula, data = regdf).fit()
print(reg.summary())
print(reg.get_margeff().summary())

         Current function value: 0.065167
         Iterations: 35
                          Probit Regression Results                           
Dep. Variable:                  mtJob   No. Observations:                14972
Model:                         Probit   Df Residuals:                    14947
Method:                           MLE   Df Model:                           24
Date:                Sun, 12 Jan 2020   Pseudo R-squ.:                  0.1554
Time:                        22:28:18   Log-Likelihood:                -975.67
converged:                      False   LL-Null:                       -1155.2
Covariance Type:            nonrobust   LLR p-value:                 1.723e-61
                        coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------
Intercept            -5.6105    396.283     -0.014      0.989    -782.312     771.091
hs[T.True]            0.0475      0.095     

//anaconda3/envs/econ/lib/python3.7/site-packages/statsmodels/base/model.py:512: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)


       Probit Marginal Effects       
Dep. Variable:                  mtJob
Method:                          dydx
At:                           overall
                       dy/dx    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------
hs[T.True]            0.0016      0.003      0.502      0.615      -0.005       0.008
college[T.True]      -0.0005      0.004     -0.112      0.911      -0.009       0.008
technical[T.True]    -0.0048      0.003     -1.409      0.159      -0.011       0.002
married[T.True]      -0.0098      0.003     -3.857      0.000      -0.015      -0.005
indig[T.True]         0.0092      0.003      3.176      0.001       0.004       0.015
male                  0.0325      0.003     10.821      0.000       0.027       0.038
edad                 -0.0007    8.3e-05     -8.057      0.000      -0.001      -0.001
hh_size              -0.0006      0.000     -1.279      0.201      -0.002 

#### (B9)

In [40]:
reg = smf.probit('mtJob ~ years_ed' + dg_formula, data = regdf).fit()
print(reg.summary())
print(reg.get_margeff().summary())

Optimization terminated successfully.
         Current function value: 0.074797
         Iterations 9
                          Probit Regression Results                           
Dep. Variable:                  mtJob   No. Observations:                12103
Model:                         Probit   Df Residuals:                    12096
Method:                           MLE   Df Model:                            6
Date:                Sun, 12 Jan 2020   Pseudo R-squ.:                  0.1221
Time:                        22:28:18   Log-Likelihood:                -905.27
converged:                       True   LL-Null:                       -1031.2
Covariance Type:            nonrobust   LLR p-value:                 1.714e-51
                      coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------
Intercept          -1.8100      0.157    -11.538      0.000      -2.117      -1.503
married[T.True

#### (B10)

In [41]:
reg = smf.probit('mtJob ~ years_ed' + dg_formula + ent_dummies_formula, data = regdf).fit()
print(reg.summary())
print(reg.get_margeff().summary())

         Current function value: 0.072435
         Iterations: 35
                          Probit Regression Results                           
Dep. Variable:                  mtJob   No. Observations:                12103
Model:                         Probit   Df Residuals:                    12080
Method:                           MLE   Df Model:                           22
Date:                Sun, 12 Jan 2020   Pseudo R-squ.:                  0.1498
Time:                        22:28:19   Log-Likelihood:                -876.68
converged:                      False   LL-Null:                       -1031.2
Covariance Type:            nonrobust   LLR p-value:                 1.862e-52
                      coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------
Intercept          -4.5002     22.526     -0.200      0.842     -48.651      39.651
married[T.True]    -0.3119      0.082     -3.813  

//anaconda3/envs/econ/lib/python3.7/site-packages/statsmodels/base/model.py:512: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)


       Probit Marginal Effects       
Dep. Variable:                  mtJob
Method:                          dydx
At:                           overall
                     dy/dx    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------
married[T.True]    -0.0114      0.003     -3.747      0.000      -0.017      -0.005
indig[T.True]       0.0133      0.003      3.812      0.000       0.006       0.020
years_ed            0.0003      0.000      1.478      0.139   -8.34e-05       0.001
male                0.0354      0.003     10.252      0.000       0.029       0.042
edad               -0.0007      0.000     -6.727      0.000      -0.001      -0.000
hh_size            -0.0005      0.001     -0.814      0.416      -0.002       0.001
ent_3               0.0820      0.823      0.100      0.921      -1.530       1.694
ent_5               0.0912      0.823      0.111      0.912      -1.521       1.704
ent_9   

#### (B11)

In [42]:
reg = smf.probit('mtJob ~ years_ed_impute' + dg_formula, data = regdf).fit()
print(reg.summary())
print(reg.get_margeff().summary())

Optimization terminated successfully.
         Current function value: 0.066944
         Iterations 9
                          Probit Regression Results                           
Dep. Variable:                  mtJob   No. Observations:                14972
Model:                         Probit   Df Residuals:                    14965
Method:                           MLE   Df Model:                            6
Date:                Sun, 12 Jan 2020   Pseudo R-squ.:                  0.1324
Time:                        22:28:19   Log-Likelihood:                -1002.3
converged:                       True   LL-Null:                       -1155.2
Covariance Type:            nonrobust   LLR p-value:                 4.409e-63
                      coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------
Intercept          -1.7593      0.147    -11.931      0.000      -2.048      -1.470
married[T.True

#### (B12)

In [43]:
reg = smf.probit('mtJob ~ years_ed_impute' + dg_formula + ent_dummies_formula, data = regdf).fit()
print(reg.summary())
print(reg.get_margeff().summary())

         Current function value: 0.065183
         Iterations: 35
                          Probit Regression Results                           
Dep. Variable:                  mtJob   No. Observations:                14972
Model:                         Probit   Df Residuals:                    14949
Method:                           MLE   Df Model:                           22
Date:                Sun, 12 Jan 2020   Pseudo R-squ.:                  0.1552
Time:                        22:28:20   Log-Likelihood:                -975.91
converged:                      False   LL-Null:                       -1155.2
Covariance Type:            nonrobust   LLR p-value:                 1.314e-62
                      coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------
Intercept          -5.7405    458.583     -0.013      0.990    -904.547     893.066
married[T.True]    -0.3002      0.076     -3.952  

//anaconda3/envs/econ/lib/python3.7/site-packages/statsmodels/base/model.py:512: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)


       Probit Marginal Effects       
Dep. Variable:                  mtJob
Method:                          dydx
At:                           overall
                     dy/dx    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------
married[T.True]    -0.0098      0.003     -3.883      0.000      -0.015      -0.005
indig[T.True]       0.0095      0.003      3.295      0.001       0.004       0.015
years_ed_impute     0.0002      0.000      1.317      0.188   -9.95e-05       0.001
male                0.0324      0.003     10.835      0.000       0.027       0.038
edad               -0.0007   8.22e-05     -8.073      0.000      -0.001      -0.001
hh_size            -0.0006      0.000     -1.161      0.246      -0.001       0.000
ent_3               0.1277     15.010      0.009      0.993     -29.290      29.546
ent_5               0.1234     15.010      0.008      0.993     -29.295      29.542
ent_9   

## Permanent Migration for Labor

In [44]:
regdf = c02_ed

regdf = pd.merge(regdf, c02_permanent, how='left', on = ['folio','ls'])
regdf["mg_perm_job"] = regdf["mg_perm_job"].fillna(0)
regdf = pd.merge(regdf, c02_hm, how='inner', on = ['folio','ls'])
regdf = pd.merge(regdf, c02_ls, how='inner', on = ['folio','ls'])

ent_dummies = pd.get_dummies(regdf["ent"])
ent_dummies.columns = ["ent_" + str(col).split(".")[0] for col in ent_dummies.columns]
regdf = pd.concat((regdf, ent_dummies), axis = 1)

regdf = regdf.drop_duplicates()

In [45]:
ent_dummies_formula = ''

for col in ent_dummies.columns:
    ent_dummies_formula += (" + %s" % col)

print(ent_dummies_formula)

 + ent_3 + ent_5 + ent_9 + ent_10 + ent_11 + ent_14 + ent_15 + ent_16 + ent_17 + ent_19 + ent_20 + ent_21 + ent_25 + ent_26 + ent_30 + ent_31


#### (C1)

In [46]:
reg = smf.probit('mg_perm_job ~ hs' + dg_formula, data = regdf).fit()
print(reg.summary())
print(reg.get_margeff().summary())

Optimization terminated successfully.
         Current function value: 0.348829
         Iterations 6
                          Probit Regression Results                           
Dep. Variable:            mg_perm_job   No. Observations:                14956
Model:                         Probit   Df Residuals:                    14949
Method:                           MLE   Df Model:                            6
Date:                Sun, 12 Jan 2020   Pseudo R-squ.:                 0.07240
Time:                        22:28:21   Log-Likelihood:                -5217.1
converged:                       True   LL-Null:                       -5624.3
Covariance Type:            nonrobust   LLR p-value:                1.219e-172
                      coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------
Intercept          -1.5559      0.068    -22.724      0.000      -1.690      -1.422
hs[T.True]    

#### (C2)

In [47]:
reg = smf.probit('mg_perm_job ~ hs' + dg_formula + ent_dummies_formula, data = regdf).fit()
print(reg.summary())
print(reg.get_margeff().summary())

Optimization terminated successfully.
         Current function value: 0.343616
         Iterations 6
                          Probit Regression Results                           
Dep. Variable:            mg_perm_job   No. Observations:                14956
Model:                         Probit   Df Residuals:                    14933
Method:                           MLE   Df Model:                           22
Date:                Sun, 12 Jan 2020   Pseudo R-squ.:                 0.08626
Time:                        22:28:21   Log-Likelihood:                -5139.1
converged:                       True   LL-Null:                       -5624.3
Covariance Type:            nonrobust   LLR p-value:                4.128e-191
                      coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------
Intercept          -1.4773      0.220     -6.703      0.000      -1.909      -1.045
hs[T.True]    

#### (C3)


In [48]:
reg = smf.probit('mg_perm_job ~ hs + college' + dg_formula, data = regdf).fit()
print(reg.summary())
print(reg.get_margeff().summary())

Optimization terminated successfully.
         Current function value: 0.348100
         Iterations 6
                          Probit Regression Results                           
Dep. Variable:            mg_perm_job   No. Observations:                14956
Model:                         Probit   Df Residuals:                    14948
Method:                           MLE   Df Model:                            7
Date:                Sun, 12 Jan 2020   Pseudo R-squ.:                 0.07433
Time:                        22:28:21   Log-Likelihood:                -5206.2
converged:                       True   LL-Null:                       -5624.3
Covariance Type:            nonrobust   LLR p-value:                2.936e-176
                      coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------
Intercept          -1.5366      0.069    -22.399      0.000      -1.671      -1.402
hs[T.True]    

#### (C4)

In [49]:
reg = smf.probit('mg_perm_job ~ hs + college' + dg_formula + ent_dummies_formula, data = regdf).fit()
print(reg.summary())
print(reg.get_margeff().summary())

Optimization terminated successfully.
         Current function value: 0.342701
         Iterations 6
                          Probit Regression Results                           
Dep. Variable:            mg_perm_job   No. Observations:                14956
Model:                         Probit   Df Residuals:                    14932
Method:                           MLE   Df Model:                           23
Date:                Sun, 12 Jan 2020   Pseudo R-squ.:                 0.08869
Time:                        22:28:22   Log-Likelihood:                -5125.4
converged:                       True   LL-Null:                       -5624.3
Covariance Type:            nonrobust   LLR p-value:                4.192e-196
                      coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------
Intercept          -1.4496      0.221     -6.566      0.000      -1.882      -1.017
hs[T.True]    

#### (C5)

In [50]:
reg = smf.probit('mg_perm_job ~ technical' + dg_formula, data = regdf).fit()
print(reg.summary())
print(reg.get_margeff().summary())

Optimization terminated successfully.
         Current function value: 0.349107
         Iterations 6
                          Probit Regression Results                           
Dep. Variable:            mg_perm_job   No. Observations:                14956
Model:                         Probit   Df Residuals:                    14949
Method:                           MLE   Df Model:                            6
Date:                Sun, 12 Jan 2020   Pseudo R-squ.:                 0.07166
Time:                        22:28:22   Log-Likelihood:                -5221.2
converged:                       True   LL-Null:                       -5624.3
Covariance Type:            nonrobust   LLR p-value:                7.602e-171
                        coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------
Intercept            -1.5226      0.068    -22.491      0.000      -1.655      -1.390
technica

#### (C6)

In [51]:
reg = smf.probit('mg_perm_job ~ technical' + dg_formula + ent_dummies_formula, data = regdf).fit()
print(reg.summary())
print(reg.get_margeff().summary())

Optimization terminated successfully.
         Current function value: 0.343816
         Iterations 6
                          Probit Regression Results                           
Dep. Variable:            mg_perm_job   No. Observations:                14956
Model:                         Probit   Df Residuals:                    14933
Method:                           MLE   Df Model:                           22
Date:                Sun, 12 Jan 2020   Pseudo R-squ.:                 0.08573
Time:                        22:28:22   Log-Likelihood:                -5142.1
converged:                       True   LL-Null:                       -5624.3
Covariance Type:            nonrobust   LLR p-value:                7.644e-190
                        coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------
Intercept            -1.4538      0.220     -6.594      0.000      -1.886      -1.022
technica

#### (C7)

In [52]:
reg = smf.probit('mg_perm_job ~ hs + college + technical' + dg_formula, data = regdf).fit()
print(reg.summary())
print(reg.get_margeff().summary())

Optimization terminated successfully.
         Current function value: 0.348017
         Iterations 6
                          Probit Regression Results                           
Dep. Variable:            mg_perm_job   No. Observations:                14956
Model:                         Probit   Df Residuals:                    14947
Method:                           MLE   Df Model:                            8
Date:                Sun, 12 Jan 2020   Pseudo R-squ.:                 0.07455
Time:                        22:28:23   Log-Likelihood:                -5204.9
converged:                       True   LL-Null:                       -5624.3
Covariance Type:            nonrobust   LLR p-value:                9.678e-176
                        coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------
Intercept            -1.5535      0.069    -22.359      0.000      -1.690      -1.417
hs[T.Tru

#### (C8)

In [53]:
reg = smf.probit('mg_perm_job ~ hs + college + technical' + dg_formula + ent_dummies_formula, data = regdf).fit()
print(reg.summary())
print(reg.get_margeff().summary())

Optimization terminated successfully.
         Current function value: 0.342662
         Iterations 6
                          Probit Regression Results                           
Dep. Variable:            mg_perm_job   No. Observations:                14956
Model:                         Probit   Df Residuals:                    14931
Method:                           MLE   Df Model:                           24
Date:                Sun, 12 Jan 2020   Pseudo R-squ.:                 0.08879
Time:                        22:28:23   Log-Likelihood:                -5124.9
converged:                       True   LL-Null:                       -5624.3
Covariance Type:            nonrobust   LLR p-value:                1.598e-195
                        coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------
Intercept            -1.4627      0.221     -6.620      0.000      -1.896      -1.030
hs[T.Tru

#### (C9)

In [54]:
reg = smf.probit('mg_perm_job ~ years_ed' + dg_formula, data = regdf).fit()
print(reg.summary())
print(reg.get_margeff().summary())

Optimization terminated successfully.
         Current function value: 0.352371
         Iterations 6
                          Probit Regression Results                           
Dep. Variable:            mg_perm_job   No. Observations:                12088
Model:                         Probit   Df Residuals:                    12081
Method:                           MLE   Df Model:                            6
Date:                Sun, 12 Jan 2020   Pseudo R-squ.:                 0.07030
Time:                        22:28:24   Log-Likelihood:                -4259.5
converged:                       True   LL-Null:                       -4581.6
Covariance Type:            nonrobust   LLR p-value:                6.736e-136
                      coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------
Intercept          -1.6700      0.080    -20.780      0.000      -1.828      -1.513
married[T.True

#### (C10)

In [55]:
reg = smf.probit('mg_perm_job ~ years_ed' + dg_formula + ent_dummies_formula, data = regdf).fit()
print(reg.summary())
print(reg.get_margeff().summary())

Optimization terminated successfully.
         Current function value: 0.348501
         Iterations 6
                          Probit Regression Results                           
Dep. Variable:            mg_perm_job   No. Observations:                12088
Model:                         Probit   Df Residuals:                    12065
Method:                           MLE   Df Model:                           22
Date:                Sun, 12 Jan 2020   Pseudo R-squ.:                 0.08052
Time:                        22:28:24   Log-Likelihood:                -4212.7
converged:                       True   LL-Null:                       -4581.6
Covariance Type:            nonrobust   LLR p-value:                8.198e-142
                      coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------
Intercept          -1.9756      0.292     -6.770      0.000      -2.547      -1.404
married[T.True

#### (C11)

In [56]:
reg = smf.probit('mg_perm_job ~ years_ed_impute' + dg_formula, data = regdf).fit()
print(reg.summary())
print(reg.get_margeff().summary())

Optimization terminated successfully.
         Current function value: 0.348854
         Iterations 6
                          Probit Regression Results                           
Dep. Variable:            mg_perm_job   No. Observations:                14956
Model:                         Probit   Df Residuals:                    14949
Method:                           MLE   Df Model:                            6
Date:                Sun, 12 Jan 2020   Pseudo R-squ.:                 0.07233
Time:                        22:28:25   Log-Likelihood:                -5217.5
converged:                       True   LL-Null:                       -5624.3
Covariance Type:            nonrobust   LLR p-value:                1.773e-172
                      coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------
Intercept          -1.5826      0.071    -22.240      0.000      -1.722      -1.443
married[T.True

#### (C12)

In [57]:
reg = smf.probit('mg_perm_job ~ years_ed_impute' + dg_formula + ent_dummies_formula, data = regdf).fit()
print(reg.summary())
print(reg.get_margeff().summary())

Optimization terminated successfully.
         Current function value: 0.343595
         Iterations 6
                          Probit Regression Results                           
Dep. Variable:            mg_perm_job   No. Observations:                14956
Model:                         Probit   Df Residuals:                    14933
Method:                           MLE   Df Model:                           22
Date:                Sun, 12 Jan 2020   Pseudo R-squ.:                 0.08631
Time:                        22:28:25   Log-Likelihood:                -5138.8
converged:                       True   LL-Null:                       -5624.3
Covariance Type:            nonrobust   LLR p-value:                3.028e-191
                      coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------
Intercept          -1.5046      0.221     -6.803      0.000      -1.938      -1.071
married[T.True